# Tensorflow Serving

- Step 1. Load the model

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
model = keras.models.load_model('/home/elegant00/Dropbox/Ml_Zoomcamp/ML/ML-Zoomcamp-Homeworks/WEEK 9/model/clothing-model.h5')

2022-01-09 21:06:37.788386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 21:06:37.863431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 21:06:37.863624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 21:06:37.865119: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [24]:
tf.saved_model.save(model,'clothing-model')

INFO:tensorflow:Assets written to: clothing-model/assets


In [3]:
!tree clothing-model

clothing-model
├── assets
├── saved_model.pb
└── variables
    ├── variables.data-00000-of-00001
    └── variables.index

2 directories, 3 files


In [9]:
! saved_model_cli show --dir clothing-model --all

fish: Unknown command: saved_model_cli
fish: 
 saved_model_cli show --dir clothing-model --all
 ^


## Mounting Docker volume:


`docker run -it --rm \
    -p 8500:8500 \
    -v "./"`


```console
-v "./clothing-model/models/clothing-model/1" 

``` 
 Volume mounting


```console
-e "MODEL_NAME='clothing-model" \
tensorflow/serving:2.7.0 
```
(Enviroment variable) - same name as folder 

Final command:

```console

docker run -it --rm \                                         (py39)  Sun 09 Jan 2022 09:24:49 PM -05
      -p 8500:8500 \
      -v "/home/elegant00/Dropbox/Ml_Zoomcamp/ML/ML-Zoomcamp-Homeworks/WEEK 10/clothing-model:/models/clothing-model/1" \
      -e MODEL_NAME="clothing-model" \
      tensorflow/serving:2.7.0

```

### Tensorflow testing :

In [5]:
%pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

     |████████████████████████████████| 4.0 MB 752 kB/s            
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.40.0
    Uninstalling grpcio-1.40.0:
      Successfully uninstalled grpcio-1.40.0
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install keras-image-helper

     |████████████████████████████████| 4.3 MB 753 kB/s            
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.4.3 requires pyparsing>=2.2.1, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [19]:
import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc


In [20]:
host='localhost:8500'

#Create local channel
channel = grpc.insecure_channel(host)
# use the channel to connect to the prediction service

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)


In [21]:
from keras_image_helper import create_preprocessor

In [24]:
preprocessor = create_preprocessor('xception',target_size=(299, 299))

In [23]:
url = 'http://bit.ly/mlbookcamp-pants'

In [37]:
X = preprocessor.from_url(url)

- Prepare request(Protobuff)

- Turn Numpy array into protobuff

In [44]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data,shape=data.shape)

In [50]:
%%capture
np_to_protobuf(X)

In [51]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'

pb_request.model_spec.signature_name='serving_default'


pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [52]:
pb_response = stub.Predict(pb_request,timeout=20.0)

In [53]:
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.87986421585083
    float_val: -4.75631046295166
    float_val: -2.359531879425049
    float_val: -1.08926522731781
    float_val: 9.903782844543457
    float_val: -2.826179027557373
    float_val: -3.648310422897339
    float_val: 3.241154909133911
    float_val: -2.612096071243286
    float_val: -4.852035045623779
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [56]:
preds = pb_response.outputs['dense_7'].float_val
preds

[-1.87986421585083, -4.75631046295166, -2.359531879425049, -1.08926522731781, 9.903782844543457, -2.826179027557373, -3.648310422897339, 3.241154909133911, -2.612096071243286, -4.852035045623779]

In [57]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [58]:
dict(zip(classes,preds))

{'dress': -1.87986421585083,
 'hat': -4.75631046295166,
 'longsleeve': -2.359531879425049,
 'outwear': -1.08926522731781,
 'pants': 9.903782844543457,
 'shirt': -2.826179027557373,
 'shoes': -3.648310422897339,
 'shorts': 3.241154909133911,
 'skirt': -2.612096071243286,
 't-shirt': -4.852035045623779}

- Next : Turning it into a Flask app

### ML Zoomcamp 10.3 - Creating a Pre-Processing Service
